In [85]:
from sklearn.metrics import ndcg_score, precision_score, recall_score
from surprise import NMF, SVD, Dataset, KNNBasic, accuracy
from surprise.model_selection import train_test_split

In [86]:
seed = 0

In [87]:
data = Dataset.load_builtin("ml-100k")

In [88]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=seed)

In [89]:
knn = KNNBasic(sim_options={"name": "pearson_baseline"}, random_state=seed)
svd = SVD(random_state=seed)
nmf = NMF(random_state=seed)

In [90]:
knn_predictions = knn.fit(trainset).test(testset)
svd_predictions = svd.fit(trainset).test(testset)
nmf_predictions = nmf.fit(trainset).test(testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [91]:
knn_predictions[:5], svd_predictions[:5], nmf_predictions[:5]

([Prediction(uid='120', iid='282', r_ui=4.0, est=3.5210151597395307, details={'actual_k': 40, 'was_impossible': False}),
  Prediction(uid='882', iid='291', r_ui=4.0, est=3.5439588060540723, details={'actual_k': 40, 'was_impossible': False}),
  Prediction(uid='535', iid='507', r_ui=5.0, est=3.8355506819136673, details={'actual_k': 38, 'was_impossible': False}),
  Prediction(uid='697', iid='244', r_ui=5.0, est=3.027281165779006, details={'actual_k': 14, 'was_impossible': False}),
  Prediction(uid='751', iid='385', r_ui=4.0, est=3.4664024390369557, details={'actual_k': 40, 'was_impossible': False})],
 [Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
  Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
  Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
  Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was

Cada elemento de los objetos de predicción contiene el id del usuario y de la película, el rating real y la predicción calculada. Por ejemplo, en la tercera, el rating real es un 5 y la predicción es un 3.8, convertido en 4, por lo que consideraría la pelicula irrelevante, cuando realmente tiene una buena valoración.

In [95]:
k = 10
for method, predictions in {
    "knn": knn_predictions,
    "svd": svd_predictions,
    "nmf": nmf_predictions,
}.items():
    predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:k]
    rmse = accuracy.rmse(predictions, verbose=False)
    X = [int(prediction.r_ui > 4) for prediction in predictions]
    y = [int(prediction.est > 4) for prediction in predictions]
    precision = precision_score(X, y)
    recall = recall_score(X, y)
    ndcg = ndcg_score([X], [y])
    print(
        f"Method: {method}, RMSE: {rmse}, Precision: {precision}, Recall: {recall}, NDCG: {ndcg}"
    )

Method: knn, RMSE: 2.1213203435596424, Precision: 0.4, Recall: 1.0, NDCG: 0.7094859686180036
Method: svd, RMSE: 0.5477225575051661, Precision: 0.7, Recall: 1.0, NDCG: 0.8742418504625449
Method: nmf, RMSE: 1.0, Precision: 0.8, Recall: 1.0, NDCG: 0.9194081433290348


El recall de 1 en los resultados indica que no hay falsos negativos, pero esto se debe a elegir tan solo el top k, evitando estos falsos negativos. La precisión nos dice los falsos positivos, es decir, peliculas valoradas como relevantes que no lo son en realidad. A más precisión, menos falsos positivos. El NDCG indica como de bien están ordenados los resultados segun la relevancia. Se puede observar como el mejor modelo es el NMF.